In [1]:
from KGLE_Dataset import *
import torch
from torch import nn
from torch.utils.data import DataLoader

In [2]:
torch.manual_seed(42)

In [3]:
train_set = KGLE_Dataset("./data/processed_train.csv")
train_load = DataLoader(dataset=train_set,
                        batch_size=1024,
                        num_workers=8,
                        shuffle=True)

valid_set = KGLE_Dataset("./data/processed_valid.csv")
valid_load = DataLoader(dataset=valid_set,
                        batch_size=99999999,
                        num_workers=8,
                        shuffle=True)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
model = nn.Sequential(
    nn.Linear(10, 1000),
    nn.ReLU(),
    nn.Linear(1000, 1000),
    nn.ReLU(),
    nn.Linear(1000, 1000),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1000, 1000),
    nn.ReLU(),
    nn.Linear(1000, 500),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(500, 200),
    nn.ReLU(),
    nn.Linear(200, 1)
).to(device)

In [5]:
lossfn = nn.MSELoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [6]:
def validate():
    loss = torch.tensor([0]).to(device, dtype=torch.float32)
    with torch.no_grad():
        for i, (entry, target) in enumerate(valid_load):
            entry = entry.to(device, dtype=torch.float32)
            target = target.to(device, dtype=torch.float32)
            preds = model(entry).to(device, dtype=torch.float32)
            loss += lossfn(preds, target)
    return loss /(i + 1)

In [7]:
def train(num_iter: int):
    for epoch in range(num_iter):
        for i, (entry, target) in enumerate(train_load):
            entry = entry.to(device, dtype=torch.float32)
            target = target.to(device, dtype=torch.float32)
            preds = model(entry).to(device, dtype=torch.float32)

            optimizer.zero_grad()
            loss = lossfn(preds, target)
            loss.backward()
            optimizer.step()
            if (i + 1) % 10 == 0 or i == 0:
                valid_loss = validate()
                print(f"[Epoch: {epoch + 1}]\t[Iter: {i + 1}]\t[RMSE: {torch.sqrt(loss)}]\t[RMSE Validation: {torch.sqrt(valid_loss).item()}]")